In [46]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import csv

## Loading data

In [116]:
#load the data 
def load_csv_data(data_path, sub_sample=False):
    """Loads data and returns y (class labels), tX (features) and ids (event ids)"""
    y = np.genfromtxt(data_path, delimiter=",", skip_header=1, dtype=str, usecols=1)
    x = np.genfromtxt(data_path, delimiter=",", skip_header=1)
    ids = x[:, 0].astype(np.int)
    input_data = x[:, 2:]

    yb = np.ones(len(y))
    yb[np.where(y == "b")] = -1

    # sub-sample
    if sub_sample:
        yb = yb[::50]
        input_data = input_data[::50]
        ids = ids[::50]

    return yb, input_data, ids

def cleanning(data):
    toDelete = []
    
    
                
    mean = np.mean(data, axis=0)
    var = np.var(data, axis=0)
    taille = len(data)
    taille2 = len(data[0])
    
    for i in range(taille):
        for j in range(taille2):
            if(data[i][j] == -999.0):
                toDelete = toDelete + [j]
                
    return np.delete(data, toDelete, axis=1), toDelete

#standardize the data
def standardize(x):

    mean_x = np.mean(x, axis=0)
    x = x - mean_x
    
    std_x = np.std(x, axis=0)
    x = x / std_x
    return x, mean_x, std_x

#create a csv submission
def create_csv_submission(ids, y_pred, name):

    with open(name, "w") as csvfile:
        fieldnames = ["Id", "Prediction"]
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({"Id": int(r1), "Prediction": int(r2)})
            
#Cross-validation implementation

def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold.
    
    Args:
        y:      shape=(N,)
        k_fold: K in K-fold, i.e. the fold num
        seed:   the random seed

    Returns:
        A 2D array of shape=(k_fold, N/k_fold) that indicates the data indices for each fold
    """
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval] for k in range(k_fold)]
    return np.array(k_indices)



In [13]:
import numpy as np
x = np.array([[3.0,2.0,4.0,5.0,7.0,7.0,7.0,7.0],[3.0,2.0,4.0,5.0,7.0,7.0,7.0,7.0],[3.0,2.0,4.0,5.0,7.0,7.0,7.0,7.0]])

((x[:,3])[x[:,3] == 5]) = 2

print(x)

[[3. 2. 4. 2. 7. 7. 7. 7.]
 [3. 2. 4. 2. 7. 7. 7. 7.]
 [3. 2. 4. 2. 7. 7. 7. 7.]]


In [119]:
train_data = load_csv_data('train.csv', sub_sample=True)
#test_data = load_csv_data('test.csv', sub_sample=False)

/tmp/ipykernel_43521/851914117.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ids = x[:, 0].astype(np.int)


In [121]:
print(train_data[1].shape)

(5000, 30)


In [122]:
#train_data[4] is ut of range

In [123]:
#x, mean_x, std_x = standardize(train_data[1])
# 1st column is 'y'
# 2nd column is 'X'
# 3rd column is 'ids'

In [124]:
train_data_cleanned, y_clean = cleanning(train_data[1], train_data[0])
train_data_std1, mean1, std1 = standardize(train_data_cleanned) #see to what it corresponds
#train_data_std2, mean2, std2 = standardize(train_data[2]) #see to what it corresponds

TypeError: cleanning() takes 1 positional argument but 2 were given

In [125]:

def build_model_data(height, weight):   
    
    y = weight
    x = height
    num_samples = len(y)
    tx = np.c_[np.ones(num_samples), x]
    return y, tx


def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]

In [126]:
def Multi_Model(y, tx, initial_w, max_iters, gamma):
    tx0 = []
    tx1 = []
    tx2 = []
    tx3 = []
    taille = len(tx)
    for i in range(taille):
        if(tx[i][22]==0):
            tx0 = tx0 + [tx[i]]
        else:
            if(tx[i][22]==1):
                tx1 = tx1 + [tx[i]]
            else:
                if(tx[i][22]==2):
                    tx2 = tx2 + [tx[i]]
                else:
                    if(tx[i][22]==3):
                        tx3 = tx3 + [tx[i]]
    
    return np.array(tx0), np.array(tx1), np.array(tx2), np.array(tx3)

In [127]:
tx0, tx1, tx2, tx3 = Multi_Model(y, train_data[1], np.ones(len(tx[0])), 10000, 0.2)
print(train_data_cleanned.shape)
print(np.array(tx0).shape)
print(np.array(tx1).shape)
print(np.array(tx2).shape)
print(np.array(tx3).shape)

(195520, 30)
(1985, 30)
(1540, 30)
(1043, 30)
(432, 30)


In [128]:
tx0_bis, y_clean1 = cleanning(tx0)
tx1_bis, y_clean2 = cleanning(tx1)
tx2_bis, y_clean3 = cleanning(tx2)
tx3_bis, y_clean4 = cleanning(tx3)

In [131]:
print(tx0_bis.shape)
print(tx1_bis.shape)
print(tx2_bis.shape)
print(tx3_bis.shape)

y_clean = y

(1985, 19)
(1540, 22)
(1043, 29)
(432, 29)


In [133]:
import implementations

_, tx = build_model_data(tx0_bis, tx0_bis)

print(tx.shape)

#_, tc = build_model_data(test_data[1], test_data[1])

y = (y_clean + 1)/2

print(max(y))
print(min(y))
    
loss, w = implementations.logistic_regression(y, tx, np.ones(len(tx[0])), 10000, 0.2)
print()
loss1, w1 = implementations.reg_logistic_regression(y, tx, 1, np.ones(len(tx[0])), 10000, 0.2)
print()
loss2, w2 = implementations.mean_squared_error_gd(y, tx, np.ones(len(tx[0])), 10000, 0.02)
print()
loss3, w3 = implementations.mean_squared_error_sgd(y, tx, np.ones(len(tx[0])), 10000, 0.002)
print()
w4, mse = implementations.least_squares(y, tx)
print("ok")
w5, mse = implementations.ridge_regression(y, tx, 0.03)

#form K subgroups randomly
k_fold = 10
seed = 0
k_indices = build_k_indices(y, k_fold, seed)

#Pattern with method
rmse_tr_tmp = []
rmse_te_tmp = []
w_tr_tmp = []
w_te_tmp = []
for k in range(k_fold):
    # Put the kth group in test
    te_indice = k_indices[k]
    tr_indice = k_indices[~(np.arange(k_indices.shape[0]) == k)]
    tr_indice = tr_indice.reshape(-1)
    y_te = y[te_indice]
    y_tr = y[tr_indice]
    x_te = tx[te_indice]
    x_tr = tx[tr_indice]
    loss_tr, w_tr = implementations.logistic_regression(y_tr, x_tr, np.ones(len(x_tr[0])), 10000, 0.2)
    loss_te, w_te = implementations.logistic_regression(y_te, x_te, np.ones(len(x_te[0])), 10000, 0.2)
    rmse_tr_tmp.append(loss_tr)
    rmse_te_tmp.append(loss_te)
    w_tr_tmp.append(w_tr)
    w_te_tmp.append(w_te)
rmse_tr = np.mean(rmse_tr_tmp, axis=0)
rmse_te = np.mean(rmse_te_tmp, axis=0)
wTR = np.mean(w_tr_tmp, axis=0)
wTE = np.mean(w_te_tmp, axis=0)

    
    
predi = implementations.sigmoid(tx @ w)
predi_t = np.sign(predi - 0.5)

predi1 = implementations.sigmoid(tx @ w1)
predi_t1 = np.sign(predi1 - 0.5)

predi2 = tx @ w2
predi_t2 = np.sign(predi2 - 0.5)

predi3 = tx @ w3
predi_t3 = np.sign(predi3 - 0.5)

predi4 = tx @ w4
predi_t4 = np.sign(predi4 - 0.5)

predi5 = tx @ w5
predi_t5 = np.sign(predi5 - 0.5)

predi6 = tx @ wTR
predi_t6 = np.sign(predi6 - 0.5)

predi7 = tx @ wTE
predi_t7 = np.sign(predi7 - 0.5)


(1985, 20)
[1.]
[0.5]


ValueError: shapes (20,1985) and (195520,1985) not aligned: 1985 (dim 1) != 195520 (dim 0)

In [61]:
#test Newton
import implementations

#careful about the dimensions of y and w! (N/D,1) instead of (N/D,)!

_, tx = build_model_data(train_data_std1, train_data_std1)
w = np.zeros((tx.shape[1], 1))

y = np.array([(y_clean + 1)/2]).T

#print(tx.shape, y.shape, w.shape)

loss8, w8 = implementations.logistic_regression_newton_method(y, tx, 0.1, w, 100, 1)
print(loss8, w8)

predi8 = tx @ w8
predi_t8 = np.sign(predi8 - 0.5)

MemoryError: Unable to allocate 285. GiB for an array with shape (195520, 195520) and data type float64

In [ ]:
test = 0

for i in range(len(predi_t)):
    if(predi_t[i] != y_clean[i]):
        test += 1

print("test 0 : " + str(1 - test/len(predi_t)))

test = 0

for i in range(len(predi_t1)):
    if(predi_t1[i] != y_clean[i]):
        test += 1

print("test 1 : " + str(1 - test/len(predi_t1)))


test = 0

for i in range(len(predi_t2)):
    if(predi_t2[i] != y_clean[i]):
        test += 1

print("test 2 : " + str(1 - test/len(predi_t2)))


test = 0

for i in range(len(predi_t3)):
    if(predi_t3[i] != y_clean[i]):
        test += 1

print("test 3 : " + str(1 - test/len(predi_t3)))


test = 0

for i in range(len(predi_t4)):
    if(predi_t4[i] != y_clean[i]):
        test += 1

print("test 4 : " + str(1 - test/len(predi_t4)))


test = 0

for i in range(len(predi_t5)):
    if(predi_t5[i] != y_clean[i]):
        test += 1

print("test 5 : " + str(1 - test/len(predi_t5)))

test = 0

for i in range(len(predi_t6)):
    if(predi_t6[i] != y_clean[i]):
        test += 1

print("test 6 : " + str(1 - test/len(predi_t6)))

test = 0

for i in range(len(predi_t7)):
    if(predi_t7[i] != y_clean[i]):
        test += 1

print("test 7 : " + str(1 - test/len(predi_t7)))




test = 0

for i in range(len(predi_t8)):
    if(predi_t8[i] != y_clean[i]):
        test += 1

print("test 8 : " + str(1 - test/len(predi_t8)))